# PPO
---

### 1. Import the Necessary Packages

In [1]:
import torch
%matplotlib inline

from model.ppo_parallel import PPO
from model.network import ActorCritic
from model.environments import LunarContinuous
from logger import WandbSummaryWritter 

### 2. Instantiate the Model

Setup the hyperparameters in the code cell below.

In [2]:
# hyperparameters = {
#     'timesteps_per_batch': 1024 ,                # Number of timesteps to run per batch
#     'max_timesteps_per_episode': 1200,           # Max number of timesteps per episode
#     'n_updates_per_iteration': 5,                # Number of times to update actor/critic per iteration
#     'lr': 2.5e-4 ,                                # Learning rate of actor optimizer
#     'gamma': 0.95,                               # Discount factor to be applied when calculating Rewards-To-Go
#     'clip': 0.2                                 # Recommended 0.2, helps define the threshold to clip the ratio during SGA
# }
hyperparameters = {'gamma': 0.999, 'lr_gamma': 0.995,
                   'max_timesteps_per_episode': 1600,
							'clip_range': 0.2, 'lr': 0.005 }

misc_hyperparameters = {
    'num_workers': 8  ,
    'seed': None 
}

Initialise wandb session in the code cell below.

In [3]:
logger = WandbSummaryWritter(project='lunar', config =hyperparameters)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: pmsaraiva2712 (pmsaraiva2712-tum). Use `wandb login --relogin` to force relogin


Initialise the model fo the desired timestamps. Alternatively can specify a checkpoint to continue training.

In [4]:
checkpoint = 'ppo_parallel_checkpoints/charmed-armadillo-108/ppo_policy_960.pth'
LOAD_MODEL = False

ppo = PPO(logger, **hyperparameters, **misc_hyperparameters)

if LOAD_MODEL:
    
    env = LunarContinuous().make_environment()
    model = ActorCritic(env.observation_space.shape[0], env.action_space.shape[0])
    model.load_state_dict(torch.load(checkpoint))



### 3. Train the Model

Train model for specified amount of timestamps.

In [5]:
total_timesteps_to_train =  1_000_000
ppo.train(total_timesteps_to_train)


-------------------- Iteration #1 --------------------
Average Episodic Length: 122.1
Average Episodic Return: -256.5
Average Loss: 0.00234
Timesteps So Far: 4884
Iteration took: 11.75 secs
Current learning rate: 0.004876243765609375
------------------------------------------------------


-------------------- Iteration #2 --------------------
Average Episodic Length: 126.37
Average Episodic Return: -236.2
Average Loss: -0.00024
Timesteps So Far: 9686
Iteration took: 11.23 secs
Current learning rate: 0.004755550652328859
------------------------------------------------------


-------------------- Iteration #3 --------------------
Average Episodic Length: 124.36
Average Episodic Return: -133.21
Average Loss: -0.00098
Timesteps So Far: 14536
Iteration took: 9.26 secs
Current learning rate: 0.004637844844091639
------------------------------------------------------


-------------------- Iteration #4 --------------------
Average Episodic Length: 131.24
Average Episodic Return: -137.84
A

KeyboardInterrupt: 

### 4. Evaluate the Model

Run multiple episodes from pretrained model.

In [ ]:
ppo.test()

KeyboardInterrupt: 